# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

scooter rider injured in hit and run 

https://www.denver7.com/news/local-news/denver-woman-recovering-from-serious-injuries-after-hit-and-run-while-riding-scooter

incident_id = 2023182255

In [3]:
geocoded_location = 'Federal Blvd and 14th Ave, Denver, CO'

In [4]:
lat_lon = cda.geocode_location(geocoded_location)

# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.1

query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

nearby['reported_date_local'] = nearby['reported_date_local'].dt.tz_localize('America/Denver')

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int) #.to_clipboard()

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,38,1,0,1,0
2014,33,0,0,2,0
2015,36,1,1,5,0
2016,35,0,3,3,0
2017,38,1,2,9,0
2018,27,0,2,5,0
2019,23,1,1,2,0
2020,18,0,0,2,0
2021,22,0,4,4,1


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,316,7,17,40,2


In [7]:
display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
    , 'distance_feet'
    , 'incident_id'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [8]:
nearby[display_cols].tail(20)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle_ind,pedestrian_ind,sbi,fatality,distance_feet,incident_id
296,2023-03-24 20:48:00-06:00,TRAF - ACCIDENT - FATAL,N FEDERAL BLVD / W 14TH AVE,0.0,1.0,False,True,40.909350,2023153590
297,2023-03-31 12:57:00-06:00,TRAF - ACCIDENT,1400 BLOCK N FEDERAL BLVD,0.0,0.0,False,False,331.811781,2023165694
298,2023-04-16 13:52:00-06:00,TRAF - ACCIDENT,N FEDERAL BLVD / W 14TH AVE,0.0,0.0,False,False,40.909350,2023196515
299,2023-04-16 21:05:00-06:00,TRAF - ACCIDENT - FATAL,1300 BLOCK N FEDERAL BLVD,0.0,1.0,False,True,250.617150,2023197148
300,2023-04-21 17:42:00.000001-06:00,TRAF - ACCIDENT - HIT & RUN,N FEDERAL BLVD / W 14TH AVE,0.0,0.0,False,False,40.909350,2023206835
301,2023-05-04 18:32:00-06:00,TRAF - ACCIDENT - HIT & RUN,1400 N FEDERAL BLVD,0.0,0.0,False,False,245.014749,2023233280
302,2023-05-22 23:41:00-06:00,TRAF - ACCIDENT - HIT & RUN,N FEDERAL BLVD / W 14TH AVE,0.0,0.0,False,False,40.909350,2023269623
303,2023-05-31 16:21:00-06:00,TRAF - ACCIDENT,1400 BLOCK N FEDERAL BLVD,0.0,0.0,False,False,331.811781,2023287333
304,2023-06-05 01:05:00-06:00,TRAF - ACCIDENT - HIT & RUN,3105 W AVONDALE DR,0.0,0.0,False,False,455.224451,2023296578
305,2023-08-19 01:10:00-06:00,TRAF - ACCIDENT - HIT & RUN,W 14TH AVE / N GROVE ST,0.0,0.0,False,False,357.250838,2023446731


In [9]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
folium.Circle(location=lat_lon, weight=2, radius=radius_miles*1609.34).add_to(this_map)
this_map.fit_bounds(
    [nearby[['geo_lat', 'geo_lon']].min().values.tolist()
     , nearby[['geo_lat', 'geo_lon']].max().values.tolist()]
)
this_map

In [10]:
# Frequency of crashes
num_years = 3

most_recent_timestamp = cda.most_recent_crash_timestamp()
start_timestamp = most_recent_timestamp - pd.DateOffset(years=num_years)
days_in_time_range = (most_recent_timestamp - start_timestamp).days

nearby_in_time_range = nearby[nearby.reported_date_local > start_timestamp].copy()

days_between_crashes = days_in_time_range / len(nearby_in_time_range)

print(
    f'The location "{geocoded_location}" has had {len(nearby_in_time_range)} traffic crashes'
    , f'within a {radius_miles} mile radius'
    , f'over the last {num_years} years, an average of {days_between_crashes:.1f} days between crashes.'
    , f'In that time, there were {nearby_in_time_range.fatality.sum()} deadly crashes'
    , f'and {nearby_in_time_range.sbi.sum()} crashes causing a serious bodily injury.'
    , f'There were {(nearby_in_time_range.pedestrian_ind > 0).sum()} crashes involving a pedestrian'
    , f'and {(nearby_in_time_range.bicycle_ind > 0).sum()} crashes involving a bicyclist.'
)

The location "Federal Blvd and 14th Ave, Denver, CO" has had 67 traffic crashes within a 0.1 mile radius over the last 3 years, an average of 16.3 days between crashes. In that time, there were 3 deadly crashes and 7 crashes causing a serious bodily injury. There were 10 crashes involving a pedestrian and 2 crashes involving a bicyclist.


In [11]:
cda.conn.dispose()